In [13]:
# ─────────────────────────────
# 1. Standard library (já vem com o Python)
# ─────────────────────────────
import json
from pathlib import Path
import itertools  # se for gerar folds manualmente, por ex.
import random  # sementes

# ─────────────────────────────
# 2. Terceiros – instalação via pip/venv
# ─────────────────────────────
# ↳ dados
import pandas as pd  # leitura CSV
from datasets import Dataset, DatasetDict, load_dataset

# ↳ modelagem e métricas
import numpy as np
# import torch
# from transformers import (
#     AutoTokenizer,
#     AutoModelForTokenClassification,
#     Trainer,
#     TrainingArguments,
# )
# from seqeval.metrics import f1_score  # F1 para NER
# from sklearn.model_selection import KFold, train_test_split

from sklearn.model_selection import KFold

# # (opcional) tracking e visualização
# import mlflow  # ou tensorboard
# import matplotlib.pyplot as plt

In [2]:
JSON_PATH = "data/geocorpus-v2.json"

In [4]:
with open(JSON_PATH, encoding="utf-8") as f:
    geo_json = json.load(f)

records_geo = [
    {"tokens": item["tokens"], "ner_tags": item["ner_tokens"]} for item in geo_json
]

In [5]:
records_geo

[{'tokens': ['na',
   'escala',
   'de',
   'tempo',
   'geológico',
   ',',
   'o',
   'lopingiano',
   'é',
   'a',
   'época',
   'do',
   'período',
   'permiano',
   'da',
   'era',
   'paleozoica',
   'do',
   'éon',
   'fanerozoico',
   'que',
   'está',
   'compreendida',
   'entre',
   '260',
   'milhões',
   'e',
   '400',
   'mil',
   'anos',
   'e',
   '251',
   'milhões',
   'de',
   'anos',
   'atrás',
   ',',
   'aproximadamente',
   '.'],
  'ner_tags': ['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-epoca',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B-periodo',
   'O',
   'O',
   'B-era',
   'O',
   'O',
   'B-eon',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O']},
 {'tokens': ['o',
   'lopingiano',
   'constitui',
   'a',
   'subdivisão',
   'posterior',
   'do',
   'permiano',
   'superior',
   '.'],
  'ner_tags': ['O',
   'B-epoca',
   'O',
   'O',
   

In [6]:
geocorpus_full = Dataset.from_list(records_geo)
print(geocorpus_full)
print(geocorpus_full[0])

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 5272
})
{'tokens': ['na', 'escala', 'de', 'tempo', 'geológico', ',', 'o', 'lopingiano', 'é', 'a', 'época', 'do', 'período', 'permiano', 'da', 'era', 'paleozoica', 'do', 'éon', 'fanerozoico', 'que', 'está', 'compreendida', 'entre', '260', 'milhões', 'e', '400', 'mil', 'anos', 'e', '251', 'milhões', 'de', 'anos', 'atrás', ',', 'aproximadamente', '.'], 'ner_tags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-epoca', 'O', 'O', 'O', 'O', 'O', 'B-periodo', 'O', 'O', 'B-era', 'O', 'O', 'B-eon', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}


In [7]:
split_1 = geocorpus_full.train_test_split(test_size=0.2, seed=42)
geo_train = split_1["train"]
geo_temp = split_1["test"]

split_2 = geo_temp.train_test_split(test_size=0.5, seed=42)
geo_val = split_2["train"]
geo_test = split_2["test"]

print(len(geo_train), len(geo_val), len(geo_test))

4217 527 528


In [9]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
indices = list(range(len(geocorpus_full)))
folds = []
for train_idx, val_idx in kf.split(indices):
    folds.append((geocorpus_full.select(train_idx), geocorpus_full.select(val_idx)))

In [14]:
# ---------- GeoCorpus completo ----------
N = len(geocorpus_full)
indices = np.arange(N)

# ---------- K-Fold só para VALIDAÇÃO ----------
kf = KFold(n_splits=5, shuffle=True, random_state=42)

geo_val_folds = [
    geocorpus_full.select(val_idx)  # só o conjunto de validação
    for _, val_idx in kf.split(indices)
]

# checar tamanhos
for i, ds in enumerate(geo_val_folds, 1):
    print(f"Fold {i}: {len(ds)} exemplos de validação")

Fold 1: 1055 exemplos de validação
Fold 2: 1055 exemplos de validação
Fold 3: 1054 exemplos de validação
Fold 4: 1054 exemplos de validação
Fold 5: 1054 exemplos de validação
